In [1]:
# Load packages we need
import sys
import os
import time

import numpy as np
import pandas as pd
import sklearn

import tensorflow as tf
from tensorflow import keras

from keras.datasets import fashion_mnist, cifar10, imdb

from sklearn.model_selection import train_test_split

# import layers and callbacks we may use (may not be a complete list)
from keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.models import Model, Sequential
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
from keras.applications import MobileNetV2
from keras import regularizers
from keras.optimizers import Adam

from tensorflow.python.keras.utils import layer_utils

from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 14})

# Load the TensorBoard notebook extension
#%load_ext tensorboard


# Let's check our software versions
print('------------')
print('### Python version: ' + __import__('sys').version)
print('### NumPy version: ' + np.__version__)
print('### Scikit-learn version: ' + sklearn.__version__)
print('### Tensorflow version: ' + tf.__version__)
print('------------')

def var_exists(var_name):
    return (var_name in globals() or var_name in locals())

------------
### Python version: 3.11.7 | packaged by Anaconda, Inc. | (main, Dec 15 2023, 18:05:47) [MSC v.1916 64 bit (AMD64)]
### NumPy version: 1.26.4
### Scikit-learn version: 1.4.2
### Tensorflow version: 2.16.1
------------


In [2]:
# Path to the directory containing the CSV file and image directory
data_dir = 'data/'
csv_file = os.path.join(data_dir, 'train.csv')
image_dir = os.path.join(data_dir, 'train_images/')

# Load the CSV file
df = pd.read_csv(csv_file)

# Display the first few rows of the DataFrame to understand its structure
print(df.head())

# Filter out filenames that don't have corresponding images
existing_filenames = []
for filename in df['image_id']:
    if os.path.exists(os.path.join(image_dir, filename)):
        existing_filenames.append(filename)

# Filter the DataFrame to include only existing filenames
df = df[df['image_id'].isin(existing_filenames)]

# Extract image filenames and labels from the DataFrame
filenames = df['image_id']
labels = df['label']

# Define the mapping of labels to disease names
disease_map = {
    0: "Cassava Bacterial Blight (CBB)",
    1: "Cassava Brown Streak Disease (CBSD)",
    2: "Cassava Green Mottle (CGM)",
    3: "Cassava Mosaic Disease (CMD)",
    4: "Healthy"
}

# Convert integer labels to one-hot encoded vectors
num_classes = len(disease_map)
labels = to_categorical(labels, num_classes=num_classes)

# Split the data into training and validation sets
train_filenames, val_filenames, train_labels, val_labels = train_test_split(
    filenames, labels, test_size=0.2, random_state=42)

# Define a function to load and preprocess images
def load_and_preprocess_image(image_path):
    # Load image from file
    img = load_img(image_path, target_size=(224, 224))
    # Convert image to numpy array
    img_array = img_to_array(img)
    # Preprocess the image (e.g., normalization)
    img_array /= 255.0  # Normalize pixel values to [0, 1]
    return img_array

# Load and preprocess training images
X_train = np.array([load_and_preprocess_image(os.path.join(image_dir, filename)) for filename in train_filenames])
y_train = train_labels

# Load and preprocess validation images
X_val = np.array([load_and_preprocess_image(os.path.join(image_dir, filename)) for filename in val_filenames])
y_val = val_labels

# Check the shapes of the training and validation data
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of y_val:", y_val.shape)

         image_id  label
0  1000015157.jpg      0
1  1000201771.jpg      3
2   100042118.jpg      1
3  1000723321.jpg      1
4  1000812911.jpg      3
Shape of X_train: (12180, 224, 224, 3)
Shape of y_train: (12180, 5)
Shape of X_val: (3045, 224, 224, 3)
Shape of y_val: (3045, 5)


In [3]:
def create_compile_cnn(input_shape=[224, 224, 3], num_outputs=5, verbose=False):
    model = Sequential(name='CIFAR-10--CNN')
    
    model.add(Conv2D(16, kernel_size=(3, 3), input_shape=input_shape, activation='relu', 
                     padding='same', strides=(1, 1), kernel_initializer='lecun_uniform', name='conv1'))
    model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', 
                     padding='same', strides=(1, 1), kernel_initializer='lecun_uniform', name='conv2'))
    model.add(MaxPooling2D((2, 2), name='MaxPool1'))
    
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', 
                     padding='same', strides=(1, 1), kernel_initializer='lecun_uniform', name='conv3'))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', 
                     padding='same', strides=(1, 1), kernel_initializer='lecun_uniform', name='conv4'))
    model.add(MaxPooling2D((2, 2), name='MaxPool2'))
    
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', 
                     padding='same', strides=(1, 1), kernel_initializer='lecun_uniform', name='conv5'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', 
                     padding='same', strides=(1, 1), kernel_initializer='lecun_uniform', name='conv6'))
    model.add(MaxPooling2D((2, 2), name='MaxPool3'))
    
    model.add(Flatten(name='flatten'))
    
    model.add(Dense(64, activation='relu', kernel_initializer='lecun_uniform', name='dense1'))
    model.add(Dropout(0.25, name='drop1'))
    
    model.add(Dense(32, activation='relu', kernel_initializer='lecun_uniform', name='dense2'))
    model.add(Dropout(0.25, name='drop2'))
    
    model.add(Dense(num_outputs, activation='softmax', name='output'))
    
    opt = keras.optimizers.Adam(learning_rate=0.001)
    
    if verbose:
        model.summary()
    
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model

In [4]:
model = create_compile_cnn(verbose=True)

c:\Users\yashh\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "CIFAR-10--CNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 224, 224, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 224, 224, 16)   │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MaxPool1 (MaxPooling2D)         │ (None, 112, 112, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3 (Conv2D)                  │ (None, 112, 112, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv4 (Conv2D)                  │ (None, 112, 112, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MaxPool2 (MaxPooling2D)         │ (None, 56, 56, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv5 (Conv2D)                  │ (None, 56, 56, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv6 (Conv2D)                  │ (None, 56, 56, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MaxPool3 (MaxPooling2D)         │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ (None, 64)             │     3,211,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop1 (Dropout)                 │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense2 (Dense)                  │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop2 (Dropout)                 │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,285,653 (12.53 MB)

 Trainable params: 3,285,653 (12.53 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
early_stop_cb = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
max_epochs = 15
batch_size = 64
    
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=max_epochs, batch_size=batch_size, 
                         shuffle=True, callbacks=[early_stop_cb])

Epoch 1/15
191/191 ━━━━━━━━━━━━━━━━━━━━ 94s 458ms/step - accuracy: 0.5893 - loss: 1.2245 - val_accuracy: 0.6095 - val_loss: 1.0771
Epoch 2/15
191/191 ━━━━━━━━━━━━━━━━━━━━ 69s 358ms/step - accuracy: 0.6431 - loss: 0.9959 - val_accuracy: 0.6204 - val_loss: 1.0599
Epoch 3/15
191/191 ━━━━━━━━━━━━━━━━━━━━ 67s 351ms/step - accuracy: 0.6437 - loss: 0.9703 - val_accuracy: 0.6322 - val_loss: 0.9553
Epoch 4/15
191/191 ━━━━━━━━━━━━━━━━━━━━ 70s 367ms/step - accuracy: 0.6614 - loss: 0.9111 - val_accuracy: 0.6542 - val_loss: 0.9205
Epoch 5/15
191/191 ━━━━━━━━━━━━━━━━━━━━ 68s 353ms/step - accuracy: 0.6696 - loss: 0.8567 - val_accuracy: 0.6545 - val_loss: 0.9049
Epoch 6/15
191/191 ━━━━━━━━━━━━━━━━━━━━ 77s 402ms/step - accuracy: 0.6951 - loss: 0.7685 - val_accuracy: 0.6447 - val_loss: 0.9337
Epoch 7/15
191/191 ━━━━━━━━━━━━━━━━━━━━ 69s 357ms/step - accuracy: 0.7358 - loss: 0.6631 - val_accuracy: 0.6489 - val_loss: 0.9642
Epoch 8/15
191/191 ━━━━━━━━━━━━━━━━━━━━ 67s 351ms/step - accuracy: 0.7657 - loss: 0